In [ ]:
# Imports
import tensorflow as tf
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
import os
import numpy as np

In [ ]:
tf.set_random_seed(0)

files = os.listdir("../data/")
files = map((lambda x: "../data/" + x), files)

# A vector of filenames.
filenames = tf.constant(files)

In [ ]:
filename_queue = tf.train.string_input_producer(filenames)

reader = tf.WholeFileReader()
key, value = reader.read(filename_queue)

images = tf.image.decode_jpeg(value, channels=3)

# initialize
init = tf.global_variables_initializer()

In [ ]:
def plotImages(num, imgs):
    # Start populating the filename queue.
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)

    for i in range(num): #length of your filename list
        image = imgs.eval() 
        
        print(image.shape)
        plt.imshow(image.squeeze(), cmap=plt.cm.gray, interpolation='nearest')
        plt.show()

        coord.request_stop()
        coord.join(threads)

In [ ]:
def prepareImages(imgs):
    sz_imgs = tf.image.resize_images(imgs, [512, 512], tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    bw_imgs = tf.image.rgb_to_grayscale(sz_imgs)
    
    return sz_imgs, bw_imgs

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    original_images, prep_images = prepareImages(images)
    #plotImages(len(files), prep_images)
    image_batch, label_batch = tf.train.batch([prep_images, original_images], batch_size=10)
    print image_batch
    print label_batch